# Leveraging Gen AI for SAT Prep

In [1]:
!pip uninstall accelerate --yes

Found existing installation: accelerate 1.2.1
Uninstalling accelerate-1.2.1:
  Successfully uninstalled accelerate-1.2.1


I’ll start with sharing my motivation for creating this notebook. When I was preparing for SAT mid-2023, I struggled to find extra study material for the new Digital SAT (launched in Spring of 2023), especially for the reading and writing section, which is tougher to crack compared to the math section. I had an idea: why not ask ChatGPT? But to my surprise, the SAT-style questions it generated were lackluster and I knew that I could be the only one, leading me to dive deeper into researching how I can leverage Gen AI for SAT prep.

I ended up figuring out an approach to leverage Llama 3 Instruct models for building my proficiency in word in context type of questions. Through that I was able to build my vocabulary, which is the critical aspect in scoring well in the reading and writing section. I scored 760 out of 800 in my first SAT attempt, which is 99th percentile among all test takers.

You don't need a GPU device to run this notebook. Just CPU based Colab instance is sufficient.

We'll start with installing transformers and torch libraries

In [2]:
!pip install transformers
!pip install torch
!pip install accelerate>=0.26.0

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
import transformers
import torch
import accelerate
print(transformers.__version__)
print(torch.__version__)
print(accelerate.__version__)

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.48.0
2.5.1
1.2.1


To run this notebook, you will need a Hugging Face (HF) token as Llama models are gated and require users to accept Meta’s usage terms. To get a token, you will have to provide your contact information in the HF model page (https://huggingface.co/meta-llama/Llama-3.1-70B-Instruct), accept the terms, and you will receive an email once the access has been approved. Followed by that, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab by navigating to the "Secrets" tab in the left panel and creating a new secret named "HF_TOKEN" and paste your Hugging Face token as the value. Restart you session thereafer.

We use Llama 3.1-8B model due to resource restrictions. If you have a powerful machine, you can leverage the 70B model.

In [4]:
#model_id = "meta-llama/Meta-Llama-3.1-70B-Instruct"
#model_id = "meta-llama/Llama-3.1-8B"
model_id="meta-llama/Meta-Llama-3-8B-Instruct"

### SAT Vocabulary Dataset

I created a SAT vocabulary dataset containing 500+ words. You can download the csv from my Git Repository. I used a CSV fromat that can be converted to Hugging Face Datasets if need be.

In [5]:
import pandas as pd
import random
from random import randrange
vocab_df = pd.read_csv('sat_vocab.csv')
print("Sample word: {} ".format(vocab_df['word'][0]))

Sample word: Abate 


### SAT style pragraph Genre

I created a Genre dataset that can be used to instruct the model to create context based on a given Genre.

In [6]:
genre_df = pd.read_csv('sat_genre.csv')
print("Sample genre: {} ".format(genre_df['genre'][0]))

Sample genre: Emergence of Homo sapiens 


## Download Llama Model
The following step will download the model weights and will take about 8 - 10 mins.

In [7]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

2025-01-12 22:19:22.649154: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736720362.667072    3818 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736720362.672564    3818 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.08it/s]
Device set to use cuda:0


We'll create a few utility function to running inference against the model, mask the SAT word, and parse the output.

We'll use Zero-short prompting to teach the model the output format we would like. Meta's Llama models are capable of following instructions and producing responses without having previously seen an example of a task. Prompting without examples is called "zero-shot prompting".

In [168]:
paragraph_length=150
replacement_mask='________________'
#function for generating a paragraph given a SAT word and Genre
def generate_paragraph(word, genre, debug):
  #We'll use the following zero-shot format:
  # Question: .... Answer is:...."
  zero_shot="Question: Generate a paragraph on The history of figure skating with the word deterimental in it. The Answer is: Figure skating has a rich and storied history that spans over 4,000 years. The earliest evidence of figure skating dates back to the 12th century in Scandinavia, where it was a popular mode of transportation during the winter months. However, the sport as we know it today began to take shape in the 18th century, with the establishment of the first skating clubs in Europe. The introduction of artificial ice rinks in the late 19th century revolutionized the sport, allowing for more precise and controlled movements. Unfortunately, the rapid growth and commercialization of figure skating in the 20th century had a deterimental effect on the sport, leading to a focus on flashy jumps and spins over technical skill and art. "
  instruction = zero_shot + "Question: Generate a paragraph on {} with the word {} in it. The Answer is:".format(genre, word)
  if debug: print("instruction: {}".format(instruction))
  output = pipeline(
    instruction,
    max_new_tokens=paragraph_length,
  )
  return output

#function for maskign the SAT word from the generated paragraph
def mask_word(word, paragraph):
  return paragraph.replace(word, replacement_mask)

#function to create a list of answer choices that has the correct work and three incorrect word
def get_answer_choices(word):
  ans_choices = [word.lower()]
  for i in range(20):
    temp_ans = vocab_df['word'][randrange(vocab_df.shape[0])]
    if temp_ans not in ans_choices:
      ans_choices.append(temp_ans.lower())
    if(len(ans_choices) >= 4):
      break
  random.shuffle(ans_choices)
  return ans_choices

#this function calls all other functions
def run_prep (word, genre, debug):
  para = generate_paragraph(word, genre, debug)
  para = mask_word(word, para)
  return [para, get_answer_choices(word)]

def run_test(debug=False):
  sat_word = (vocab_df['word'][randrange(vocab_df.shape[0])]).lower()
  sat_genre = genre_df['genre'][randrange(genre_df.shape[0])]
  if debug: print("word: {}, genre: {}".format(sat_word, sat_genre))
  para = generate_paragraph(sat_word, sat_genre, debug)
  if debug: print("output: {}".format(para))
  #split the response by "Question: " text that we use in zero-shot
  para = para[0]['generated_text'].split("Question:")[2]
  para = mask_word(sat_word, para)
  if debug: print("masked output: {}".format(para))
  #further split the response by "The Answer is: " text that we use in zero-shot
  para = para.split("The Answer is:")
  answer_choices = get_answer_choices(sat_word)
  formatted_ans_choices = "Answer choices: 1/ {}, 2/ {}, 3/ {}, 4/ {}".format(answer_choices[0],answer_choices[1],answer_choices[2],answer_choices[3])
  formatted_correct_answer = "Correct Answer: {}".format(sat_word)
  return [para[1], formatted_ans_choices, formatted_correct_answer]

## Creating Test Questions

In [174]:
output = run_test(False)
para = output[0]
answer_choices = output[1]
correct_answer = output[2]
print("Question: {}".format(para))
print()
print("Answer choices: {}".format(answer_choices))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question:  The dissolution of the Soviet Union in 1991 marked the end of an era of ________________ rule in Eastern Europe. For nearly seven decades, the Soviet Union had maintained a tight grip on its satellite states, suppressing dissent and opposition through a combination of propaganda, censorship, and force. The collapse of the Soviet Union led to a wave of democratization across the region, as countries such as Poland, Hungary, and Czechoslovakia began to transition to more open and inclusive political systems. The dissolution of the Soviet Union also had far-reaching consequences for international relations, as the global balance of power shifted and new alliances were formed. 

Answer choices: Answer choices: 1/ clemency, 2/ inept, 3/ reprieve, 4/ authoritarian


In [175]:
print(correct_answer)

Correct Answer: authoritarian


## Evaluation
We evaluate the validity of the test questions by checking how many of the generated texts contain the SAT word we had asked for in each request. More test can be done but for now we are keeping it light.

In [185]:
results=[]
eval_count=20
valid_questions_count=0
for i in range(eval_count):
  print("Generating question: {}".format(i+1))
  output = run_test()
  if replacement_mask in output[0]:
      valid_questions_count += 1
  results.append([output[0],output[1],output[2]])
print("Number of eval questions {}; Valid questions generation {}".format(eval_count, valid_questions_count))
for i in range(len(results)):
    print(results[i])
    print("#################################")
    

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generating question: 1


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generating question: 2


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Generating question: 18


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generating question: 19


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generating question: 20
Number of eval questions 20; Valid questions generation 19
[' The emergence of Homo sapiens is a complex and still debated topic in the field of paleoanthropology. Fossil evidence suggests that Homo sapiens evolved from Homo heidelbergensis, a species that lived in Africa around 600,000 years ago. However, the exact timeline and geographical location of this emergence is still a topic of much debate. Some scientists argue that Homo sapiens may have evolved in multiple regions, while others propose a single origin theory. It is also worth noting that the search for answers has led some researchers to ________________ into the realm of genetics, where the study of DNA has provided valuable insights into human evolution. Despite these uncertainties, it is clear that Homo sapiens emerged as a distinct species around 300,000', 'Answer choices: 1/ irresolute, 2/ assiduous, 3/ commendable, 4/ digress', 'Correct Answer: digress']
#################################
[' The

## Limitations

Following are some of the limitations of this SAT question genreation approach. I'll further work to make it perfect.
* Ending paragraphs gracefully
* Limiting Extraneous Tokens
* Checking for hallucination. The model hallucinates quite a bit